## In this notebook, information about basics of dataset transform is presented.

Complete list of PyTorch built-in transforms can be found in the documentation here:
https://pytorch.org/vision/stable/transforms.html

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [2]:
# Our dataset from the previous tutorial - we will work to extend the class to support our transforms and write our own transform classes

class WineDatasetOld(Dataset):
    
    def __init__(self):
        # data loading
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.x = xy[:, 1:]
        self.y = xy[:, [0]] # n_samples, 1
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        # dataset[0]
        return self.x[index], self.y[index]
    def __len__(self):
        # size
        return self.n_samples

In [3]:
# Our old wine custom datset, now modified to support transforms

class WineDataset(Dataset):
    
    def __init__(self, transform=None ):
        # data loading
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        self.x = xy[:, 1:]
        self.y = xy[:, [0]] # n_samples, 1
        
        self.transform = transform
        
    def __getitem__(self, index):
        # dataset[0]
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        # size
        return self.n_samples

In [4]:
# Our own transform class, which converts data from NumPy NDArrays to PyTorch tensors.

class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
# Our own transform class, which multiplies the inputs by a given factor.
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

In [5]:
# Now, we create a sequential transform
seq_transform = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])

In [6]:
# And create one dataset using the transform, and one not using, to show the difference.
no_transform_dataset = WineDatasetOld()

first_data = no_transform_dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features)

transform_dataset = WineDataset(transform=seq_transform)

first_data = transform_dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03])
